In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

pd.set_option('display.max_columns', None)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
target = pd.read_csv("../data/target_train.csv")
plavki_train = pd.read_csv("../data/plavki_train.csv")
plavki_test = pd.read_csv("../data/plavki_test.csv")

In [4]:
from plavki import process_plavki

In [39]:
data, num_features, cat_features = process_plavki(plavki_train, plavki_test, bow_count=20)

In [40]:
data[num_features + cat_features]

,dayofmonth,hour,duration,truncated_NMZ,st_diff_is_zero,dayofweek,plavka_TIPE_GOL,plavka_TIPE_FUR,plavka_NAPR_ZAD,bow_.z0,bow_/Э,bow_/ЭТ,bow_01/,bow_1/Э,bow_2/Э,bow_2M/,bow_3/Э,bow_3пс,bow_C2/,bow_M/Э,bow_P.z,bow_SC2,bow_SP.,bow_z01,bow_Ст3,bow_ЭТ,bow_пс/,bow_с/Э,bow_т3п
0,1,3,2579.0,С255,1,4,0,цилиндрическая,МНЛЗ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,4,4004.0,С255,1,4,0,цилиндрическая,МНЛЗ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,5,2904.0,Ст3пс/Э,1,4,0,цилиндрическая,Изл,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1,1
3,1,6,3291.0,other,1,4,0,цилиндрическая,Изл,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,7,2895.0,SC2M/ЭТ,1,4,0,цилиндрическая,МНЛС,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2838,29,17,2837.0,other,1,1,2,цилиндрическая,МНЛЗ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2839,29,18,2810.0,other,1,1,2,цилиндрическая,МНЛЗ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2840,29,18,3498.0,other,1,1,2,цилиндрическая,МНЛЗ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2841,29,20,2438.0,other,1,1,2,цилиндрическая,МНЛЗ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [13]:
target.isna().sum()

NPLV    0
TST     0
C       2
dtype: int64

In [55]:
data = data.loc[~data["NPLV"].isin([511156, 512299])]

In [130]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler


column_transformer = ColumnTransformer([
    ('encoding', OrdinalEncoder(), cat_features),
    ('scaling', StandardScaler(), num_features),
    #('scaling', 'passthrough', num_features)
])

train = data.loc[data["is_train"] == 1, cat_features + num_features]
test = data.loc[data["is_train"] == 0, cat_features + num_features]

column_transformer.fit(train)
X_train = column_transformer.transform(train)
X_test = column_transformer.transform(test)

In [72]:
from sklearn.model_selection import train_test_split

In [131]:
train_ids = data[data["is_train"] == 1]["NPLV"]
test_ids = data[data["is_train"] == 0]["NPLV"]

In [132]:
c_train = target.set_index("NPLV").loc[train_ids]["C"]
tst_train = target.set_index("NPLV").loc[train_ids]["TST"]

In [63]:
# X_train, X_eval, c_train, c_eval, tst_train, tst_eval = train_test_split(X_train, c_train, tst_train, 
#                                                                         shuffle=True, test_size=0.2)

In [133]:
c_train.shape

(2061,)

In [90]:
tst_train.shape

(2061,)

In [91]:
X_train.shape

(1054, 29)

In [134]:
X_train, X_eval = X_train[:int(X_train.shape[0] * 0.8)], X_train[int(X_train.shape[0] * 0.8):]
c_train, c_eval = c_train[:int(c_train.shape[0] * 0.8)], c_train[int(c_train.shape[0] * 0.8):]
tst_train, tst_eval = tst_train[:int(tst_train.shape[0] * 0.8)], tst_train[int(tst_train.shape[0] * 0.8):]

In [135]:
from xgboost import XGBRegressor

xgb_c_model = XGBRegressor().fit(X_train, c_train)
xgb_tst_model = XGBRegressor().fit(X_train, tst_train)

In [136]:
c_pred, tst_pred = c_model.predict(X_eval), tst_model.predict(X_eval)

In [137]:
def metric(c_true, tst_true, c_pred, tst_pred):

    delta_c = np.abs(c_true - c_pred)
    hit_rate_c = np.int64(delta_c < 0.02)

    delta_t = np.abs(tst_true - tst_pred)
    hit_rate_t = np.int64(delta_t < 20)

    N = c_pred.shape[0]
    
    return np.sum(hit_rate_c) / N, np.sum(hit_rate_t) / N, np.sum(hit_rate_c + hit_rate_t) / 2 / N

In [138]:
metric(c_eval, tst_eval, c_pred, tst_pred)

(0.48668280871670705, 0.5302663438256658, 0.5084745762711864)

In [148]:
from sklearn.svm import SVR

c_model = SVR().fit(X_train, c_train)
tst_model = SVR().fit(X_train, tst_train)

In [149]:
c_pred, tst_pred = c_model.predict(X_eval), tst_model.predict(X_eval)
metric(c_eval, tst_eval, c_pred, tst_pred)

(0.18401937046004843, 0.576271186440678, 0.3801452784503632)

In [139]:
xgb_test_c = xgb_c_model.predict(X_test)
xgb_test_t = xgb_tst_model.predict(X_test)

In [150]:
ri_test_c = c_model.predict(X_test)
ri_test_t = tst_model.predict(X_test)

In [144]:
sample = pd.read_csv("../data/sample_submission.csv")

In [151]:
sample["TST"] = ri_test_t
sample["C"] = xgb_test_c

In [153]:
sample.to_csv("baseline.csv", index=False)

In [154]:
gas_train = pd.read_csv("../data/gas_train.csv")
gas_test = pd.read_csv("../data/gas_test.csv")

In [156]:
import sys
sys.path.append("../gas")

In [157]:
from features import generate_gas_features

In [158]:
gas_feat_train = generate_gas_features(gas_train)

In [159]:
gas_feat_train

,gas_duration,gas_mean_V,gas_mean_T,gas_mean_O2,gas_mean_N2,gas_mean_H2,gas_mean_CO2,gas_mean_CO,gas_mean_AR,gas_mean_T фурмы 1,gas_mean_T фурмы 2,gas_mean_O2_pressure,gas_std_V,gas_std_T,gas_std_O2,gas_std_N2,gas_std_H2,gas_std_CO2,gas_std_CO,gas_std_AR,gas_std_T фурмы 1,gas_std_T фурмы 2,gas_std_O2_pressure,gas_sum_volume_O2,gas_sum_volume_N2,gas_sum_volume_H2,gas_sum_volume_CO2,gas_sum_volume_CO,gas_sum_volume_AR,gas_mean_volume_O2,gas_mean_volume_N2,gas_mean_volume_H2,gas_mean_volume_CO2,gas_mean_volume_CO,gas_mean_volume_AR,gas_std_volume_O2,gas_std_volume_N2,gas_std_volume_H2,gas_std_volume_CO2,gas_std_volume_CO,gas_std_volume_AR
NPLV,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
510008,2560,216789.292999,506.912198,8.080398,60.971011,0.300348,13.646908,16.236455,0.801776,0.000000,0.000000,13.356058,5369.811888,273.329244,8.601033,19.843291,0.798206,7.340789,20.627802,0.189599,0.000000,0.000000,0.292023,4.502321e+07,3.391881e+08,1.624003e+06,7.575220e+07,8.913852e+07,4.458630e+06,17587.192761,132495.366168,634.376020,29590.704477,34819.734247,1741.652526,18701.426346,43818.901201,1655.726824,15946.086198,44086.505490,425.909341
510009,3949,217054.421867,375.840941,10.985339,64.784377,0.163313,11.761720,11.466482,0.838266,0.000000,0.000000,13.950163,4250.571350,291.238099,10.616286,16.819827,0.538952,8.095644,18.080320,0.167612,0.000000,0.000000,1.003299,9.485353e+07,5.562788e+08,1.364358e+06,1.004032e+08,9.705018e+07,7.193678e+06,24019.631951,140865.726324,345.494449,25424.967838,24575.886359,1821.645526,23278.991109,37231.830524,1123.023713,17501.298735,38641.706115,373.312144
510010,2871,215606.942311,489.881937,8.745518,62.633599,0.312984,12.723079,14.755786,0.828832,0.000000,0.000000,14.022366,6659.001169,301.143710,9.489587,19.181862,0.797407,8.108025,19.036043,0.167892,0.000000,0.000000,0.330850,5.488448e+07,3.886485e+08,1.874444e+06,7.840161e+07,9.005883e+07,5.138755e+06,19116.849917,135370.437027,652.889050,27308.119240,31368.453349,1789.883300,20947.742464,42260.955051,1623.914332,17333.228492,40336.007001,376.987465
510011,3261,218908.844905,439.273874,9.016227,62.605303,0.100366,13.566362,13.771099,0.835434,0.000000,0.000000,14.250926,5341.347771,267.257197,8.395936,17.444834,0.215926,7.790133,18.094440,0.194226,0.000000,0.000000,0.665478,6.478924e+07,4.477267e+08,7.091111e+05,9.674182e+07,9.717645e+07,5.974200e+06,19867.905305,137297.364374,217.452049,29666.305531,29799.586702,1832.014654,18607.672797,38883.156660,468.388798,17089.798747,39012.937295,437.916480
510012,2860,217981.805452,478.608197,8.470485,62.286450,0.442823,13.643787,14.273926,0.802074,0.000000,0.000000,14.259779,5757.933597,282.254884,9.054424,18.477454,0.878580,8.633744,18.366216,0.182802,0.000000,0.000000,0.652421,5.328600e+07,3.894694e+08,2.698631e+06,8.476063e+07,8.769451e+07,5.013289e+06,18631.467155,136178.120683,943.577316,29636.583246,30662.417509,1752.898361,19987.724321,41474.251736,1859.532540,18623.947548,39404.426535,419.124607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512318,3060,208248.303329,439.574555,4.152793,53.516385,1.191883,17.997605,22.405678,0.736660,28.701842,25.883701,15.004221,7548.831043,246.178558,5.610260,21.662747,1.267843,8.394836,20.068151,0.219879,5.081189,0.080459,0.990635,2.659664e+07,3.426617e+08,7.449856e+06,1.149322e+08,1.408978e+08,4.706967e+06,8691.713694,111980.943607,2434.593391,37559.535747,46045.018769,1538.224385,11749.972138,45978.755732,2509.652248,17805.931661,40458.211385,468.885956
512319,3803,206951.068996,336.801224,8.228796,62.478096,0.696861,12.629975,15.130581,0.831568,28.272125,26.229685,14.582193,6563.914573,264.291825,6.208800,22.759382,1.188099,7.192062,22.684423,0.232025,4.975858,0.087734,0.983222,6.564858e+07,4.959340e+08,5.250390e+06,9.879792e+07,1.147826e+08,6.588681e+06,17262.314047,130405.990406,1380.591525,25978.942183,30182.131880,1732.495661,13041.372039

# ААОАОАУАОАОЫЫЫ

In [162]:
sys.path.append("../chugun")

from features_chugun import process_data

In [204]:
target = pd.read_csv("../data/target_train.csv")
plavki_train = pd.read_csv("../data/plavki_train.csv")
plavki_test = pd.read_csv("../data/plavki_test.csv")

gas_train = pd.read_csv("../data/gas_train.csv")
gas_test = pd.read_csv("../data/gas_test.csv")

chugun_train = pd.read_csv("../data/chugun_train.csv")
chugun_test = pd.read_csv("../data/chugun_test.csv")

In [205]:
params = {
    "chugun": {},
    "plavki": {}
}

plavki_data, plavki_num_features, plavki_cat_features = process_plavki(plavki_train, plavki_test, **params["plavki"])
gas_train, gas_test = generate_gas_features(gas_train), generate_gas_features(gas_test)
chugun_data, chugun_num_features, chugun_cat_features = process_data(chugun_train, chugun_test, **params["chugun"])
gas_num_features = list(gas_train.columns)
gas_cat_features = []

In [222]:
train = target[["NPLV"]][~target["C"].isna()]
test = sample[["NPLV"]]

train = train.join(gas_train, on="NPLV")
train = train.join(plavki_data.set_index("NPLV")[plavki_num_features + plavki_cat_features], on="NPLV")
train = train.join(chugun_data.set_index("NPLV")[chugun_num_features + chugun_cat_features], on="NPLV")

In [223]:
test = test.join(gas_test, on="NPLV")
test = test.join(plavki_data.set_index("NPLV")[plavki_num_features + plavki_cat_features], on="NPLV")
test = test.join(chugun_data.set_index("NPLV")[chugun_num_features + chugun_cat_features], on="NPLV")

In [224]:
num_features = gas_num_features + plavki_num_features + chugun_num_features
cat_features = gas_cat_features + plavki_cat_features + chugun_cat_features

In [219]:
train = train[~train["NPLV"].isin(bad_target)]

2

In [227]:
train.set_index("NPLV", inplace=True)
test.set_index("NPLV", inplace=True)

In [228]:
y = target.set_index("NPLV").loc[train.index]

# Ыээыэыэыэы

In [230]:
target = pd.read_csv("../data/target_train.csv")
sample = pd.read_csv("../data/sample_submission.csv")

plavki_train = pd.read_csv("../data/plavki_train.csv")
plavki_test = pd.read_csv("../data/plavki_test.csv")

gas_train = pd.read_csv("../data/gas_train.csv")
gas_test = pd.read_csv("../data/gas_test.csv")

chugun_train = pd.read_csv("../data/chugun_train.csv")
chugun_test = pd.read_csv("../data/chugun_test.csv")

In [254]:
from pipeline import merge_data

params = {
    "chugun": {},
    "plavki": {"bow_count": 1}
}

train, test, y, num_features, cat_features = merge_data(
    sample, target, plavki_train, plavki_test, gas_train, gas_test, chugun_train, chugun_test, params
)

In [255]:
from pipeline import train_eval_split

x_train, x_eval, y_train, y_eval = train_eval_split(train, y)

In [263]:
from xgboost import XGBRegressor

model_c = XGBRegressor(n_estimators=20).fit(train, y["C"])
model_tst = XGBRegressor(n_estimators=20).fit(train, y["TST"])

metric(y_eval["C"], y_eval["TST"], model_c.predict(x_eval), model_tst.predict(x_eval))

(0.8498789346246973, 0.8329297820823245, 0.8414043583535109)

In [278]:
sample["TST"] = model_tst.predict(x_test)
sample["C"] = model_c.predict(x_test)

In [279]:
sample.to_csv("baseline_knn.csv", index=False)

In [252]:
import shap

In [268]:
column_transformer = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown='ignore'), cat_features),
    ('scaling', StandardScaler(), num_features)
])

In [270]:
x_train = column_transformer.fit_transform(x_train)
x_eval = column_transformer.transform(x_eval)
x_test = column_transformer.transform(test)

In [276]:
from sklearn.neighbors import KNeighborsRegressor

model_c = KNeighborsRegressor().fit(x_train, y_train["C"])
model_tst = KNeighborsRegressor().fit(x_train, y_train["TST"])

metric(y_eval["C"], y_eval["TST"], model_c.predict(x_eval), model_tst.predict(x_eval))

(0.5375302663438256, 0.559322033898305, 0.5484261501210653)

In [275]:
x_train.shape

(1648, 110)